# portfolio

> A module to model a portfolio's performance over the available time-scale including the impact of fees.

A notebook to explore different investment portfolio ideas and compare their historic returns, inclusive of fees

In [ ]:
#| default_exp portfolio

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *
import pdb
from datetime import date

In [ ]:
#| export
import pytz
from fastcore.utils import *

# Holding
A holding is an asset that can be bought. We need to track it's value and how much of it we own over time. We therefore need financial data which can be provided by yfinance for free.

Let's import the package and have a play. Remembering that when we are using `nbdev` we have to have imports in their own cells.

In [ ]:
#| export
import yfinance as yf

In [ ]:
price_history = yf.Ticker('VWELX').history(period='max', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                           interval='1d', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                           actions=False)
price_history.head()

,Open,High,Low,Close,Volume
Date,,,,,
1980-01-02 00:00:00-05:00,0.522507,0.522507,0.522507,0.522507,0
1980-01-03 00:00:00-05:00,0.520185,0.520185,0.520185,0.520185,0
1980-01-04 00:00:00-05:00,0.523088,0.523088,0.523088,0.523088,0
1980-01-07 00:00:00-05:00,0.523088,0.523088,0.523088,0.523088,0
1980-01-08 00:00:00-05:00,0.529474,0.529474,0.529474,0.529474,0


In [ ]:
price_history.index[-1]

Timestamp('2023-06-23 00:00:00-0400', tz='America/New_York')

yfinance appears to be a good option as it has price information. One concern would be what units these are but that doesn't matter as we will convert back to cash at the end.

In [ ]:
show_doc(yf.Ticker.history)

---

### log_indent_decorator.<locals>.wrapper

>      log_indent_decorator.<locals>.wrapper (*args, **kwargs)

A useful check would be to see what the holding returns if a fund doesn't exist.

In [ ]:
price_history = yf.Ticker("cats").history(period='max', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                                    interval='1d', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                                    actions=False)
price_history.shape[0]

CATS: No timezone found, symbol may be delisted


0

## Deposits

Deposits to our fund could be naturally specified as time series of deposits. A helper function is provided to support the creation of monthly deposits. It is important that these days lie on business days so we use `pd.bdate_range` However, this could be extended to different time periods.

In [ ]:
#| export
import pandas as pd
from datetime import datetime
from tzlocal import get_localzone

In [ ]:
#| export
to_datetime = lambda date_string: pytz.timezone(str(get_localzone())).localize(datetime.strptime(date_string,"%d/%m/%Y"))
def create_monthly_deposits(start:str,        # Date of the first montly deposit.
                            end:str,          # Date of the last monthly deposit
                            deposit:float):    # Value of monthly deposit
    dti = pd.bdate_range(start=to_datetime(start),end=to_datetime(end),freq='BM',tz=str(get_localzone()))
    deposits = [deposit]*len(dti)
    return pd.Series([deposit]*len(dti), index=dti, name='deposits')

We call our function as follows.

In [ ]:
monthly_deposits = create_monthly_deposits('01/01/2022','01/04/2023',10)
monthly_deposits

2022-01-31 00:00:00+00:00    10
2022-02-28 00:00:00+00:00    10
2022-03-31 00:00:00+01:00    10
2022-04-29 00:00:00+01:00    10
2022-05-31 00:00:00+01:00    10
2022-06-30 00:00:00+01:00    10
2022-07-29 00:00:00+01:00    10
2022-08-31 00:00:00+01:00    10
2022-09-30 00:00:00+01:00    10
2022-10-31 00:00:00+00:00    10
2022-11-30 00:00:00+00:00    10
2022-12-30 00:00:00+00:00    10
2023-01-31 00:00:00+00:00    10
2023-02-28 00:00:00+00:00    10
2023-03-31 00:00:00+01:00    10
Freq: BM, Name: deposits, dtype: int64

In [ ]:
monthly_deposits = create_monthly_deposits('01/01/2022','01/04/2023',10)
monthly_deposits

2022-01-31 00:00:00+00:00    10
2022-02-28 00:00:00+00:00    10
2022-03-31 00:00:00+01:00    10
2022-04-29 00:00:00+01:00    10
2022-05-31 00:00:00+01:00    10
2022-06-30 00:00:00+01:00    10
2022-07-29 00:00:00+01:00    10
2022-08-31 00:00:00+01:00    10
2022-09-30 00:00:00+01:00    10
2022-10-31 00:00:00+00:00    10
2022-11-30 00:00:00+00:00    10
2022-12-30 00:00:00+00:00    10
2023-01-31 00:00:00+00:00    10
2023-02-28 00:00:00+00:00    10
2023-03-31 00:00:00+01:00    10
Freq: BM, Name: deposits, dtype: int64

## Single Holding
First we create a portfolio consisting of a single holding that takes a deposits argument. A main potential confusion here is if the deposits predate the commencement of the fund. Let's see the oldest possible date available where we have prices for the Vanguard Life Strategy 100% fund.

In [ ]:
price_history = yf.Ticker('0P0000TKZO.L').history(period='max', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                           interval='1d', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                           actions=False)
price_history.head()

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-02 00:00:00+00:00,20677.300781,20677.300781,20677.300781,20677.300781,0
2018-01-03 00:00:00+00:00,20863.699219,20863.699219,20863.699219,20863.699219,0
2018-01-04 00:00:00+00:00,20965.300781,20965.300781,20965.300781,20965.300781,0
2018-01-05 00:00:00+00:00,21076.099609,21076.099609,21076.099609,21076.099609,0
2018-01-08 00:00:00+00:00,21060.300781,21060.300781,21060.300781,21060.300781,0


Let's create some deposits that predate these dates slightly. Try "GB00B4Q5X527.L" too.

In [ ]:
monthly_deposits = create_monthly_deposits('01/01/2017','01/04/2023',10)

What happens when we join these two tables?

In [ ]:
price_history = pd.merge(monthly_deposits.to_frame(),price_history,left_index=True,right_index=True,how='outer')
price_history

,deposits,Open,High,Low,Close,Volume
2017-01-31 00:00:00+00:00,10.0,NaN,NaN,NaN,NaN,NaN
2017-02-28 00:00:00+00:00,10.0,NaN,NaN,NaN,NaN,NaN
2017-03-31 00:00:00+01:00,10.0,NaN,NaN,NaN,NaN,NaN
2017-04-28 00:00:00+01:00,10.0,NaN,NaN,NaN,NaN,NaN
2017-05-31 00:00:00+01:00,10.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2023-06-16 00:00:00+01:00,NaN,30315.039062,30315.039062,30315.039062,30315.039062,0.0
2023-06-19 00:00:00+01:00,NaN,30194.650391,30194.650391,30194.650391,30194.650391,0.0
2023-06-20 00:00:00+01:00,NaN,30184.279297,30184.279297,30184.279297,30184.279297,0.0
2023-06-21 00:00:00+01:00,NaN,30042.669922,30042.669922,30042.669922,30042.669922,0.0


How do we want this to behave?
* When we have a portfolio we are going to be combining funds that don't start on the same dates.
* We want funds to all have the same start date - achieved by using the same deposits input.
* We fill the price_history with days.
* We could save up all deposits until the first business day, but if we've back filled fund's pricing i.e. the price is the same as when the fund opens for all preceding days, then we've effectively achieved this. We won't see spikes in the deposits/fund value to see this but the final calculation will be the same and rebalancing will be easier. This also has the marginal downside that the fees are  incorrect. I would prefer the user to just construct a portfolio with funds that have appropriate start dates than adding in lots of functionality for this edge case.

Let's start by infilling with days. This is also important to get the fees correct! 

In [ ]:
dti = pd.date_range(start=price_history.index[0],end=price_history.index[-1])
price_history = price_history.reindex(dti)

In [ ]:
price_history['deposits'] = price_history['deposits'].fillna(0)
price_history = price_history.fillna(method='bfill')
price_history

,deposits,Open,High,Low,Close,Volume
2017-01-31 00:00:00+00:00,10.0,20677.300781,20677.300781,20677.300781,20677.300781,0.0
2017-02-01 00:00:00+00:00,0.0,20677.300781,20677.300781,20677.300781,20677.300781,0.0
2017-02-02 00:00:00+00:00,0.0,20677.300781,20677.300781,20677.300781,20677.300781,0.0
2017-02-03 00:00:00+00:00,0.0,20677.300781,20677.300781,20677.300781,20677.300781,0.0
2017-02-04 00:00:00+00:00,0.0,20677.300781,20677.300781,20677.300781,20677.300781,0.0
...,...,...,...,...,...,...
2023-06-18 00:00:00+01:00,0.0,30194.650391,30194.650391,30194.650391,30194.650391,0.0
2023-06-19 00:00:00+01:00,0.0,30194.650391,30194.650391,30194.650391,30194.650391,0.0
2023-06-20 00:00:00+01:00,0.0,30184.279297,30184.279297,30184.279297,30184.279297,0.0
2023-06-21 00:00:00+01:00,0.0,30042.669922,30042.669922,30042.669922,30042.669922,0.0


In [ ]:
#| export
import warnings

In [ ]:
#| export
class Holding:
    "A holding for fund with data available on yfinance"
    def __init__(self,
                 fund:str,                      # Name of the fund
                 ticker:str,                    # Ticker symbol for the stock
                 product_cost:float,            # Sum of all fees expressed as a percentage
                 deposits):                     # Timeseries dataframe of deposits
                 
        
        self.fund = fund
        self.product_cost = product_cost
        
        # Where we've specified a timeseries where values predate the oldest time in our ticker,
        # we give a warning and then fill the deposits columns with nans and back-fill all other columns
        
        price_history = yf.Ticker(ticker).history(period='max', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                                  interval='1d', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                                  actions=False)
        price_history = price_history.tz_convert(str(get_localzone()))
        
        if price_history.shape[0] == 0:
            raise Exception(f"No fund data available for {ticker}") 
        
        #TODO: Or if there is no overlap before i.e. code assumes deposits can predate fund only.        
        if deposits.index[0] < price_history.index[0]:
            warn_msg = f"""Deposits predate initial date of {price_history.index[0]} where prices are available for {fund}. First pricing data is back-filled.""" 
            warnings.warn(warn_msg)
        elif deposits.index[0] > price_history.index[-1]:
            warn_msg = f"""{fund} closed on {price_history.index[-1]} before the first deposit on {deposits.index[0]}.""" 
            warnings.warn(warn_msg)
        
        # Trim any fund data that predates deposits
        price_history = price_history.iloc[price_history.index >= deposits.index[0]]
        
        # Join and clean
        self.history = pd.merge(deposits,price_history,left_index=True,right_index=True,how='outer')
        dti = pd.date_range(start=self.history.index[0],end=pytz.timezone(str(get_localzone())).localize(datetime.now()))
        self.history = self.history.reindex(dti)
        self.history['deposits'] = self.history['deposits'].fillna(0)
        self.history = self.history.fillna(method='bfill')
        self.history = self.history.fillna(method='ffill')
        
        self = self.compute_value()
        
    def compute_value(self):
        
        self.history['units']        = self.history['deposits']/self.history['Close']
        self.history['cum_units']    = self.history['units'].cumsum()
        self.history['cum_value']    = self.history['cum_units']*self.history['Close']
        self.history['fees'] = ((1+self.product_cost/100)**(1/365)-1)*self.history['cum_value']
        
        return self

Let's create our first holding.

In [ ]:
single_holding = Holding('Vanguard LifeStrategy 100%','0P0000TKZO.L',0.22,create_monthly_deposits('04/01/2018','01/12/2022',1000))

Our holding should have the fund name assigned.

In [ ]:
assert single_holding.fund == 'Vanguard LifeStrategy 100%'

Our holding should have a history dataframe...

In [ ]:
assert isinstance(single_holding.history, pd.DataFrame)

with certain columns...

In [ ]:
assert all([item in single_holding.history.columns for item in ['Open','High','Low','Close','deposits','units','cum_units','cum_value','fees']])

Let's check the range of our holdings.

In [ ]:
single_holding.history.head()

,deposits,Open,High,Low,Close,Volume,units,cum_units,cum_value,fees
2018-01-31 00:00:00+00:00,1000.0,20638.800781,20638.800781,20638.800781,20638.800781,0.0,0.048452,0.048452,1000.000000,0.006021
2018-02-01 00:00:00+00:00,0.0,20603.599609,20603.599609,20603.599609,20603.599609,0.0,0.000000,0.048452,998.294418,0.006011
2018-02-02 00:00:00+00:00,0.0,20399.000000,20399.000000,20399.000000,20399.000000,0.0,0.000000,0.048452,988.381070,0.005951
2018-02-03 00:00:00+00:00,0.0,19985.199219,19985.199219,19985.199219,19985.199219,0.0,0.000000,0.048452,968.331418,0.005830
2018-02-04 00:00:00+00:00,0.0,19985.199219,19985.199219,19985.199219,19985.199219,0.0,0.000000,0.048452,968.331418,0.005830


and the later dates.

In [ ]:
single_holding.history.tail()

,deposits,Open,High,Low,Close,Volume,units,cum_units,cum_value,fees
2023-06-20 00:00:00+01:00,0.0,30184.279297,30184.279297,30184.279297,30184.279297,0.0,0.0,2.456569,74149.752530,0.446440
2023-06-21 00:00:00+01:00,0.0,30042.669922,30042.669922,30042.669922,30042.669922,0.0,0.0,2.456569,73801.879387,0.444346
2023-06-22 00:00:00+01:00,0.0,30002.289062,30002.289062,30002.289062,30002.289062,0.0,0.0,2.456569,73702.681036,0.443749
2023-06-23 00:00:00+01:00,0.0,30002.289062,30002.289062,30002.289062,30002.289062,0.0,0.0,2.456569,73702.681036,0.443749
2023-06-24 00:00:00+01:00,0.0,30002.289062,30002.289062,30002.289062,30002.289062,0.0,0.0,2.456569,73702.681036,0.443749


Now, let's check we can define deposits before a fund starts.

In [ ]:
initial_deposit  = create_monthly_deposits("01/01/2018","01/02/2018",250000)
lf_ruffer_diversified = Holding("LF Ruffer Diversified","0P0001MKQK.L",1.12,initial_deposit)

/tmp/ipykernel_1041/2452922105.py:28: UserWarning: Deposits predate initial date of 2021-09-01 00:00:00+01:00 where prices are available for LF Ruffer Diversified. First pricing data is back-filled.
  warnings.warn(warn_msg)


# Portfolio Modelling
For portfolio analysis, I'd like to explore the following features:
* Modelling a porfolio consisting of multiple funds.
* Rebalancing to target allocations.
* Rebalancing at given frequency.
    
Proposed structure would be lists of fund names, tickers, allocations and ongoing charges. A deposit rate would also need to be defined. Rebalancing times could be specified as a date series.

Future functionality might include:
* Variable allocation in time.
* At present if a fund's inception date is after the first deposit then cash deposits are accumulated until the first deposit opportunity. Another possible approach would be to split the capital just between the funds that are open at a given time. Even better would be to enable concatenation of two FixedAllocationPortfolio funds... this might also be an interesting coding challenge.

A main requirement is going to be how we join funds together that have different start dates.

In [ ]:
monthly_deposits = create_monthly_deposits("01/01/2018",date.today().strftime("%d/%m/%Y"),20000/12)
lf_ruffer_diversified  = Holding("LF Ruffer Diversified","0P0001MKQK.L",1.12,initial_deposit)
lf_ruffer_diversified.history.tail()

/tmp/ipykernel_1041/2452922105.py:28: UserWarning: Deposits predate initial date of 2021-09-01 00:00:00+01:00 where prices are available for LF Ruffer Diversified. First pricing data is back-filled.
  warnings.warn(warn_msg)


,deposits,Open,High,Low,Close,Volume,units,cum_units,cum_value,fees
2023-06-20 00:00:00+01:00,0.0,100.089996,100.089996,100.089996,100.089996,0.0,0.0,2500.0,250224.990845,7.635574
2023-06-21 00:00:00+01:00,0.0,99.769997,99.769997,99.769997,99.769997,0.0,0.0,2500.0,249424.991608,7.611162
2023-06-22 00:00:00+01:00,0.0,99.589996,99.589996,99.589996,99.589996,0.0,0.0,2500.0,248974.990845,7.597430
2023-06-23 00:00:00+01:00,0.0,99.339996,99.339996,99.339996,99.339996,0.0,0.0,2500.0,248349.990845,7.578359
2023-06-24 00:00:00+01:00,0.0,99.339996,99.339996,99.339996,99.339996,0.0,0.0,2500.0,248349.990845,7.578359


In [ ]:
vanguard_life_strategy = Holding('Vanguard LifeStrategy 100%','0P0000TKZO.L',0.22,create_monthly_deposits('04/01/2018','01/12/2022',1000))
vanguard_life_strategy.history.tail()

,deposits,Open,High,Low,Close,Volume,units,cum_units,cum_value,fees
2023-06-20 00:00:00+01:00,0.0,30184.279297,30184.279297,30184.279297,30184.279297,0.0,0.0,2.456569,74149.752530,0.446440
2023-06-21 00:00:00+01:00,0.0,30042.669922,30042.669922,30042.669922,30042.669922,0.0,0.0,2.456569,73801.879387,0.444346
2023-06-22 00:00:00+01:00,0.0,30002.289062,30002.289062,30002.289062,30002.289062,0.0,0.0,2.456569,73702.681036,0.443749
2023-06-23 00:00:00+01:00,0.0,30002.289062,30002.289062,30002.289062,30002.289062,0.0,0.0,2.456569,73702.681036,0.443749
2023-06-24 00:00:00+01:00,0.0,30002.289062,30002.289062,30002.289062,30002.289062,0.0,0.0,2.456569,73702.681036,0.443749


It's natural to represent our holdings as a list...

In [ ]:
holdings = [lf_ruffer_diversified, vanguard_life_strategy]

How, many rows does each entry have?

In [ ]:
n_rows = [holding.history.shape[0] for holding in holdings]
n_rows

[1971, 1971]

So due to the definition of our holding class, we can use these interchangeably, as the dataframes are the same shape.

In [ ]:
#| export
class FixedAllocationPortfolio:
    "A collection of holdings of funds with data available on yfinance with a fixed allocation of each deposit made."
    def __init__(self,
                 fund:list,          # List of fund names
                 ticker:list,        # List of fund tickers
                 product_cost:list,  # List of fees for each fund expressed as a percentage and comprising all fees for a given fund
                 allocation:list,    # Allocation of each deposit as a fraction. Must sum to one.
                 deposits):          # Timeseries dataframe of deposits
        
        # Check fund, ticker, product_cost and allocation are all lists of equal length
        assert(all([len(input)==len(fund) for input in [ticker, product_cost, allocation]]))
        
        # Check that allocation sums to 1
        assert(sum(allocation)==1)
        
        # Record the inputs
        self.deposits = deposits
        self.allocation = allocation
        self.fund = fund
        
        # Create a holding for each fund
        self.holdings = [Holding(fund[i],ticker[i],product_cost[i],deposits*allocation[i]) for i in range(len(fund))]
               
    def rebalance(self,rebalance_dates):
        
        # We rebalance on the dates specified. If a rebalancing date is prior a funds 
        # inception date then the fund's deposit is used in the rebalancing.
        
        # It is assumed that fees are paid for with an external account and that the fees
        # are accrued daily.
        
        holdings = self.holdings
        
        # The holdings are identical so we can use any one
        matching_rows = holdings[0].history.index.get_indexer(rebalance_dates,method='nearest')
        
        # Get column references
        cum_value_idx = holdings[0].history.columns.get_loc("cum_value")
        deposit_idx = holdings[0].history.columns.get_loc("deposits")
        
        for row in matching_rows:
            
            # Compute the current allocation
            current_allocation = [holdings[i].history.iloc[row].loc['cum_value']
                                 for i in range(len(holdings))]
            
            # Compute target allocations based on asset weightings
            total_value = sum(current_allocation)
            target_allocation = [fraction*total_value for fraction in self.allocation]
            
            for i in range(len(holdings)):
                holdings[i].history.iloc[row,deposit_idx] = holdings[i].history.iloc[row,deposit_idx] \
                                                                   + (target_allocation[i] - holdings[i].history.iloc[row,cum_value_idx])
                holdings[i].compute_value()
            
        self.holdings = holdings
        return self

A method to test that everything is working would be to create a portfolio that consists of only the same funds and compare the cumulate value to a holding.

In [ ]:
fixed_allocation_portfolio = FixedAllocationPortfolio(['Vanguard LifeStrategy 100%','Vanguard LifeStrategy 100%'], \
                                                      ['0P0000TKZO.L','0P0000TKZO.L'], \
                                                      [0.22, 0.22],\
                                                      [0.5,0.5], \
                                                      create_monthly_deposits('01/01/2018','01/12/2022',1000))

We don't need to rebalance to check the cumulative value as the funds are the same.

We define a common interface to convert a selection of holdings into a single returns object. We can use the returns object to commonise plotting and visualisation of results. One thing we need is a method to determine whether we have a single holding or a fixed allocation portfolio.

In [ ]:
#| export
class Returns:
    def __init__(self,
                 name:    str ,   # Description of the returns - typically used as title
                 holdings: list):  # List of holdings
    
            self.name = name
            self.history = holdings[0].history[["deposits","cum_value","fees"]]
            for i in range(1,len(holdings)):
                self.history += holdings[i].history[["deposits","cum_value","fees"]]

Let's add methods to our `Holding` and `FixedAllocationPortfolio` classes to generate a `Returns` object.

In [ ]:
#| export
@patch
def to_returns(self:Holding):
    return Returns(self.fund,[self])

Can we generate our returns for our single holding?

In [ ]:
single_holding_returns= single_holding.to_returns()
single_holding_returns.name

'Vanguard LifeStrategy 100%'

In [ ]:
single_holding_returns.history.head()

,deposits,cum_value,fees
2018-01-31 00:00:00+00:00,1000.0,1000.000000,0.006021
2018-02-01 00:00:00+00:00,0.0,998.294418,0.006011
2018-02-02 00:00:00+00:00,0.0,988.381070,0.005951
2018-02-03 00:00:00+00:00,0.0,968.331418,0.005830
2018-02-04 00:00:00+00:00,0.0,968.331418,0.005830


Now, let's do something similar fo the `FixedAllocationPortfolio` class.

In [ ]:
#| export
@patch
def to_returns(self:FixedAllocationPortfolio):
    
    name = f"{100*self.allocation[0]}% {self.fund[0]}"
    for i in range(1,len(self.holdings)):
        name += f", {100*self.allocation[i]}% {self.fund[i]}"

    return Returns(name,self.holdings)

Similarly, does it work?

In [ ]:
fixed_allocation_portfolio_returns = fixed_allocation_portfolio.to_returns()
fixed_allocation_portfolio_returns.name

'50.0% Vanguard LifeStrategy 100%, 50.0% Vanguard LifeStrategy 100%'

In [ ]:
fixed_allocation_portfolio_returns.history.head()

,deposits,cum_value,fees
2018-01-31 00:00:00+00:00,1000.0,1000.000000,0.006021
2018-02-01 00:00:00+00:00,0.0,998.294418,0.006011
2018-02-02 00:00:00+00:00,0.0,988.381070,0.005951
2018-02-03 00:00:00+00:00,0.0,968.331418,0.005830
2018-02-04 00:00:00+00:00,0.0,968.331418,0.005830


One of the most basic things we might want to do is determine our proft....

In [ ]:
#| export
@patch
def profit(self:Returns):
    
    if self.history.iloc[-1].isnull().sum() == 3:
        return self.history['cum_value'][-2]-sum(self.history['deposits'][:-1])-sum(self.history['fees'][:-1])
    
    return self.history['cum_value'][-1]-sum(self.history['deposits'])-sum(self.history['fees'])

and let's see if this works for our single holding and our portfolio. Let's start with the the single holding.

In [ ]:
single_holding_returns.profit()

14246.28341653979

Here, we can check our `FixedAllocationPortfolio` and `Holding` classes give the same result for identical fund allocations.

In [ ]:
assert(single_holding_returns.profit()==fixed_allocation_portfolio_returns.profit())

In theory, rebalancing here should make no difference. Let's create a helper function to create a monthly rebalancing schedule...

In [ ]:
#| export
create_monthly_rebalance_dates =  lambda start, end: pd.bdate_range(start=to_datetime(start),end=to_datetime(end),freq='BM',tz = str(get_localzone()))

In [ ]:
create_monthly_rebalance_dates('01/01/2018','01/12/2022')

DatetimeIndex(['2018-01-31 00:00:00+00:00', '2018-02-28 00:00:00+00:00',
               '2018-03-30 00:00:00+01:00', '2018-04-30 00:00:00+01:00',
               '2018-05-31 00:00:00+01:00', '2018-06-29 00:00:00+01:00',
               '2018-07-31 00:00:00+01:00', '2018-08-31 00:00:00+01:00',
               '2018-09-28 00:00:00+01:00', '2018-10-31 00:00:00+00:00',
               '2018-11-30 00:00:00+00:00', '2018-12-31 00:00:00+00:00',
               '2019-01-31 00:00:00+00:00', '2019-02-28 00:00:00+00:00',
               '2019-03-29 00:00:00+00:00', '2019-04-30 00:00:00+01:00',
               '2019-05-31 00:00:00+01:00', '2019-06-28 00:00:00+01:00',
               '2019-07-31 00:00:00+01:00', '2019-08-30 00:00:00+01:00',
               '2019-09-30 00:00:00+01:00', '2019-10-31 00:00:00+00:00',
               '2019-11-29 00:00:00+00:00', '2019-12-31 00:00:00+00:00',
               '2020-01-31 00:00:00+00:00', '2020-02-28 00:00:00+00:00',
               '2020-03-31 00:00:00+01:00', '2020-0

... and check our profit does not change.

In [ ]:
fixed_allocation_portfolio_rebalanced = fixed_allocation_portfolio.rebalance(create_monthly_rebalance_dates('01/01/2018','01/12/2022'))
assert(single_holding_returns.profit() == fixed_allocation_portfolio_rebalanced.to_returns().profit())

This was quite an easy test as the funds were identical. Let see if we can use a single `FixedAllocationPortfolio` to give the same profit as two holdings where we have done a manual rebalance. Let's check how the rebalance and deposit dates align so tha we have an identical computation.

In [ ]:
create_monthly_rebalance_dates('01/01/2021','29/01/2021')

DatetimeIndex(['2021-01-29 00:00:00+00:00'], dtype='datetime64[ns, Europe/London]', freq='BM')

In [ ]:
create_monthly_deposits('30/11/2020','01/02/2021',0.8*1000)

2020-11-30 00:00:00+00:00    800.0
2020-12-31 00:00:00+00:00    800.0
2021-01-29 00:00:00+00:00    800.0
Freq: BM, Name: deposits, dtype: float64

Now, we create our desposits...

In [ ]:
holding_one_deposits = create_monthly_deposits('01/01/2018','31/12/2020',0.8*1000)

but also note the length of this array as we are going to do our final rebalance on the day that would be the next monthl deposit.

In [ ]:
rebalance_row = holding_one_deposits.shape[0]

We create our holdings with the deposit time series...

In [ ]:
holding_one = Holding("Scottish Mortgage Ord","GB00BLDYK618",0.1,holding_one_deposits)

and also note the row where our next deposit it will be. The reader should be aware that holdings track from the date of the first deposit to the present day.

In [ ]:
row = holding_one.history.index.get_indexer([to_datetime('29/01/2021')], method='nearest')[0]
row

1094

In [ ]:
holding_one.history.index[row]

Timestamp('2021-01-29 00:00:00+0000', tz='Europe/London', freq='D')

In [ ]:
holding_one_value  = holding_one.history.cum_value[row]
holding_one_value

63657.74753088376

In [ ]:
holding_two = Holding("Fundsmith Equity I Acc","GB00B41YBW71",0.1,create_monthly_deposits('01/01/2018','31/12/2020',0.2*1000))
holding_two_value  = holding_two.history.cum_value[row]
holding_two_value

8919.56425989645

In [ ]:
total_value = holding_one_value + holding_two_value
total_value

72577.31179078021

Now, let's make create two holdings for our whole duration and update the deposits to do the manual rebalance...

In [ ]:
holding_one_deposits = create_monthly_deposits('01/01/2018','31/12/2022',0.8*1000)
holding_one_deposits[rebalance_row] = holding_one_deposits[rebalance_row]+(0.8*total_value-holding_one_value)
holding_one = Holding("Scottish Mortgage Ord","GB00BLDYK618",0.1,holding_one_deposits)
holding_two_deposits = create_monthly_deposits('01/01/2018','31/12/2022',0.2*1000)
holding_two_deposits[rebalance_row] = holding_two_deposits[rebalance_row]+(0.2*total_value-holding_two_value)
holding_two = Holding("Fundsmith Equity I Acc","GB00B41YBW71",0.1,holding_two_deposits)

and do the same with our `FixedAllocationPortfolio`.

In [ ]:
multi_holding_portfolio = FixedAllocationPortfolio(["Scottish Mortgage Ord","Fundsmith Equity I Acc"],
                                                   ["GB00BLDYK618","GB00B41YBW71"],
                                                   [0.1, 0.1],
                                                   [0.8, 0.2],
                                                   create_monthly_deposits('01/01/2018','31/12/2022',1000))
rebalanced_multi_holding_portfolio = multi_holding_portfolio.rebalance(create_monthly_rebalance_dates('01/01/2021','29/01/2021'))

In [ ]:
assert(holding_one.to_returns().profit() + holding_two.to_returns().profit() - rebalanced_multi_holding_portfolio.to_returns().profit() < 0.01)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()